In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
import gradio as gr

In [3]:
load_dotenv(override=True)

openai_api_key = os.getenv("OPENAI_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
google_api_key=os.getenv("GOOGLE_API_KEY")

if openai_api_key:
    print(f"OpenAI API key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API key not set")

if anthropic_api_key:
    print(f"Anthropic API Key exits and being {anthropic_api_key[:7]}")
else:
    print("ANthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")


OpenAI API key exists and begins sk-proj-
Anthropic API Key exits and being sk-ant-
Google API Key exists and begins AIzaSyBa


In [4]:
openai=OpenAI()

anthropic_ul="https://api.anthropic.com/v1/"
gemini_url="https://generativeLanguage.googleapis.com/v1beta/openai/"

anthropic=OpenAI(base_url=anthropic_ul,api_key=anthropic_api_key)
gemini=OpenAI(base_url=gemini_url,api_key=google_api_key)

In [5]:
system_message="You are a helpful assistant"

def message_gpt(prompt):
    messages=[{"role":"system","content":system_message}, {"role":"user","content":prompt}]
    response=openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
    return response.choices[0].message.content

In [6]:
message_gpt("what is today's date?")

"Today's date is June 8, 2024."

In [7]:
## here's a simple function for user interface
def shout(text):
    print(f"Shout has been called with input {text}")
    return text.lower()

In [8]:
shout("hello")

Shout has been called with input hello


'hello'

In [ ]:
gr.Interface(fn=shout,inputs="textbox",outputs="textbox",flagging_mode="never").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Shout has been called with input hi
Shout has been called with input YYYYYYYYY


In [12]:
gr.Interface(fn=shout,inputs="textbox",outputs="textbox",flagging_mode="never").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Shout has been called with input Hi


In [13]:
gr.Interface(fn=shout,inputs="textbox",outputs="textbox",flagging_mode="never").launch(inbrowser=True,auth=("ed","bananas"))

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Shout has been called with input HI


In [ ]:
message_input = gr.Textbox(label="Your message:",info="Enter a message to be shouted", lines=7)
message_output=gr.Textbox(label="Response:",lines=8)

view = gr.Interface(fn=shout,
title="Shout",
inputs=[message_input],
outputs=[message_output],
examples=["hello","howdy"],
flagging_mode="never")

view.launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


Shout has been called with input howdy
Shout has been called with input hhhHHHH


In [15]:
message_input=gr.TextArea(label="Your Message", info="Enter a message for GPT-4.1-mini",lines=7)
message_output=gr.Textbox(label="Response:",lines=8)

view=gr.Interface(
    fn=shout,
    title="Shout",
    inputs=[message_input],
    outputs=[message_output],
    examples=["hello","howdy"],
    flagging_mode="never"

)
view.launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


In [17]:
system_message="You are a helpful assistant that responds in Markdown without code blocks"

message_intput=gr.Textbox(label="your Message:",info="Enter a message for GPT-4.1-mini", lines=7)
message_output=gr.Markdown(label="Response:")

view = gr.Interface(
    fn=message_gpt,
    title="GPT",
    inputs = [message_input],
    outputs=[message_output],
    examples=[
        "Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI Engineer"
    ],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


In [18]:
def stream_gpt(prompt):
    messages=[
        {"role":"system", "content":system_message},
        {"role":"user","content":prompt}
    ]
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=messages,
        stream=True
    )
    result =""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

Building a company broucher generator

In [19]:
system_message="""
You are an assistant that analyzes the contents of a company website landing page 
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
"""

In [23]:
from scraper import fetch_website_contents

def stream_brochure(company_name,url,model):
    yield ""
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += fetch_website_contents(url)
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result=stream_gpt(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [24]:
name_input = gr.Textbox(label="Company Name:")
url_input=gr.Textbox(label="Landing Page URL including http:// or https://")
model_selector = gr.Dropdown(["GPT","Claude"],label="select model", value ="GPT")
message_output = gr.Markdown("Response:")

view=gr.Interface(
    fn=stream_brochure,
    title="Brochure Generator",
    inputs=[name_input,url_input,model_selector],
    outputs=[message_output],
     examples=[
        ["Huggingface","https://huggingface.co","GPT"],
        ["SMX","https://smxtech.com","Claude"]
    ],
    flagging_mode = "never"

)
view.launch()


* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.
